<a href="https://colab.research.google.com/github/ctclumak/posts/blob/master/Etsy_Product_Scraping_With_Scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scrapy

In [7]:
import scrapy
import pandas as pd
from scrapy.crawler import CrawlerProcess


In [18]:
#--------Define the variables

#This holds the listing url
list_id_records = []
url_lists = []
keywords_records=[]

query = "wedding+accessories"
class EtsySpider(scrapy.Spider):
  name = "Etsy_crawler"
  allowed_domains = ['etsy.com']
  base_url = "https://www.etsy.com/search/?q=%s&ref=pagination&page=%d"
#--------creating list of urls to be scraped
  for i in range(1,5):
    url = base_url % (query,i)
    url_lists.append(url)
#------Define a Request
  def start_requests(self):
    for url in url_lists:
      return scrapy.Request(url=url,callback=self.parse)

#--------Defining a Scrapy parse
  #parse() is the Scrapy’s default callback method which is called for requests without an explicitly assigned callback
  def parse(self,response):

      # #Get the listing id
      list_id = response.css("a::attr(data-listing-id)").get_all()

      if list_id != None:
            url_product = "http://www.etsy.com/listing/" + str(list_id) +"/"
            list_id_records.append(url_product)

      for list_id in list_id_records:
        yield {
        "list_id_no": response.css("h1::attr(data-listing-id)").get(),
        "title": response.css("div.wt-mb-xs-2::text").get(),
        "seller": response.css("a.wt-text-link::text").get(),
        "price": response.css("p.wt-text-title-03 wt-mr-xs-2::text").get(),
        "total_sales": response.css("span.wt-text-caption::text").get(),
        "description": response.css("p.wt-text-body-01 wt-break-word::text").get(),
        "review": response.css("h3.wt-text-body-03::text").get(),
        "website": response.css("a.wt-text-link::attr(href)").get()   
         }
   
        keywords_records.append((list_id_no, title, seller, price, total_sales,description, review, website))
        df = pd.DataFrame(product, columns =["List_ID", "Title", "Seller","Current_Price", "Product_Sales", "Product_Detail", "Total_Review", "Website"])
        df_main = pd.DataFrame()
        df_main = pd.concat([df_main,df])
        time.sleep(random.randint(1,3))
        df_main.to_csv('etsy_products.csv', index=False, encoding='utf-8')


In [19]:
process = CrawlerProcess()
process.crawl(EtsySpider)
process.start() # the script will block here until all crawling jobs are finished

2021-03-09 13:39:31 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-09 13:39:31 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.10 (default, Feb 20 2021, 21:17:23) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2021-03-09 13:39:31 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-09 13:39:31 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-09 13:39:31 [scrapy.extensions.telnet] INFO: Telnet Password: abd7a0ef6f288ef5
2021-03-09 13:39:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-03-09 13:39:31 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downlo

In [4]:
# #This holds the listing url
# list_id_records = []
# keywords_records = []

# class EtsySpider(scrapy.Spider):
#   name = "Etsy_crawler"
#   allowed_domains = ['etsy.com']
#   base_url = "https://www.etsy.com/search/?q=%s&ref=pagination&page=%d"
#   #parse() is the Scrapy’s default callback method which is called for requests without an explicitly assigned callback
#   def parse(self,response):
#       url = response.url    
#       #This is the listing id list
#       listing_id = response.find_all("a")
#       time.sleep(random.randint(1,3))
#   #this gather listing url by listing id and adding to website address
#       for listing in listing_id:
#         list_id = (listing.get("data-listing-id"))
#         if list_id != None:
#             url_product = "http://www.etsy.com/listing/" + str(list_id) +"/"
#             list_id_records.append(url_product)

#       for list_id in list_id_records:
#         etsy_page_product = requests.get(list_id)
#         soup_product = BeautifulSoup(etsy_page_product.content, "html.parser")
#         list_id_no = soup_product.select("a.listing-link")[0].get("data-listing-id")
#         title = soup_product.find("div", {"data-component":"listing-page-title-component"}).text.strip()
#         #seller = soup_product.select(".wt-mb-xs-1 p a.wt-text-link-no-underline")[0].find("span").text.strip().replace("\n"," ")
#         price = soup_product.find("p", {"class":"wt-text-title-03 wt-mr-xs-2"}).text.strip().replace("\n"," ").replace("Price: ","").replace("+","")
#         try:
#           total_sales = soup_product.select(".wt-display-inline-flex-xs a.wt-text-link-no-underline")[0].find("span").text.strip().replace("\n"," ").replace("sales", "").strip()
#         except:
#           total_sales =0
#         description = soup_product.find("p", {"class":"wt-text-body-01 wt-break-word"}).text.strip().replace("\n"," ")
#         product_url = "http://www.etsy.com/listing/" + str(list_id_no) +"/"
#         try: 
#           review = soup_product.find("h3", {"class":"wt-text-body-03"}).text.strip().replace("\n",",").replace(",","").replace("shop","")
#         except:
#           review =0
#         #website = soup_product.select(".wt-mb-xs-1 p a.wt-text-link-no-underline")[0].get("href")
#         keywords_records.append((list_id_no, title, price, total_sales,description, product_url, review))
#         df = pd.DataFrame(keywords_records, columns =["List_ID", "Title","Current_Price", "Product_Sales", "Product_Detail", "Product_list", "Total_Review"])
#         df_main = pd.DataFrame()
#         df_main = pd.concat([df_main,df])
#         time.sleep(random.randint(1,3))
#         df_main.to_csv('etsy_products.csv', index=False, encoding='utf-8')


In [ ]:
      # for list_id in list_id_records:
      #   product = []
      #   product["list_id_no"]=response.css("h1::attr(data-listing-id)").get(),
      #   product["title"]=response.css("div.wt-mb-xs-2::text").get(),
      #   product["seller"]=response.css("a.wt-text-link::text").get(),
      #   product["price"]=response.css("p.wt-text-title-03 wt-mr-xs-2::text").get(),
      #   product["total_sales"]=response.css("span.wt-text-caption::text").get(),
      #   product["description"]=response.css("p.wt-text-body-01 wt-break-word::text").get(),
      #   product["review"]=response.css("h3.wt-text-body-03::text").get(),
      #   product["website"]=response.css("a.wt-text-link::attr(href)").get()   
      #   return product